In [1]:
from nltk.tokenize import word_tokenize
import pandas as pd
import spacy 
import matplotlib.pyplot as plt
import gzip,os,glob
import numpy as np
import pickle
import nltk
import codecs
from langdetect import detect

In [2]:
# load data from pkl file 
df = pd.read_pickle('../data/data_filtered contents.pkl')
print(df.shape)

(20247, 5)


In [3]:
# Adds new collumn, detects language of column
df['Languagereview'] = df['content'].apply(detect)

In [4]:
# delets all rows with the value "en"
df = df[df.Languagereview == "de"]

In [5]:
# resets index from 0, 4, 11, ... (random) to 0,1,2, ...
df.reset_index(inplace=True,drop=True)
print(df.shape)

(18143, 6)


In [21]:
# Removing non-alphabetical symbols in dataframe, keeping umlauts
df['content'] = df['content'].replace('[^a-zA-Z \äöüß]', '', regex=True).str.replace(' +',' ').str.strip()
# Transform umlauts
df['content'] = df['content'].str.replace(chr(252), 'ue')
df['content'] = df['content'].str.replace(chr(228), 'ae')
df['content'] = df['content'].str.replace(chr(246), 'oe')
df['content'] = df['content'].str.replace(chr(196), 'Ae')
df['content'] = df['content'].str.replace(chr(214), 'Oe')
df['content'] = df['content'].str.replace(chr(220), 'Ue')
df['content'] = df['content'].str.replace(chr(223), 'ss')
# Lowercase
df['content'] = df['content'].str.lower()
df.head()

,author,title,class,content,word_count,Languagereview,content_nltk
0,NaN,aernout mik communitas,[Kunstgeschichte],aernout mik zaehlt zu den wichtigsten kuenstle...,64,de,aernout mik communitas aernout mik zaehlt wich...
1,NaN,anton henning dt engl,NaN,die vorliegende publikation widmet sich drei o...,113,de,anton henning dt engl vorliegend publikation w...
2,specht heike,die feuchtwangers familie tradition und jüdis...,"[Sprachwissenschaften, Geschichte]",jahre geschichte deutschjuedischen buergertums...,169,de,feuchtwangers familie tradition jüdisch selbst...
3,scruton roger,grüne philosophie ein konservativer denkansatz,"[Allgemeines, Philosophie, Theologie und Relig...",der umweltschutz liegt herkoemmlich im hoheits...,101,de,grüne philosophie konservativ denkansatz umwel...
4,schlagowski heinz,technische dokumentation im maschinen und anl...,[Technik],das buch gibt dem anwender die entscheidenden ...,156,de,technische dokumentation maschinen anlagenbau ...


<h1>Combining content</h1>

In [22]:
df['title'] = df['title'].str.split('  / von').str[0]
df['title'] = df['title'].str.replace('[^\w\s]','')
df['title'] = df['title'].replace('\s+', ' ', regex=True)
df['title'] = df['title'].str.lower()

In [23]:
df['author'] = df['author'].str.split('  / von').str[0]
df['author'] = df['author'].str.replace('[^\w\s]','')
df['author'] = df['author'].replace('\s+', ' ', regex=True)
df['author'] = df['author'].str.lower()

In [24]:
df_combined_temp = df.copy()
df_combined_temp.drop(['class','word_count', 'Languagereview'],axis=1, inplace=True)
df_combined_temp.replace(np.nan, '', regex=True, inplace=True)
df_combined_temp.replace('\s+', ' ', regex=True, inplace=True)
#df_combined = df_combined_temp['author'] +" "+ df_combined_temp['title'] +" "+ df_combined_temp['content']
#df_combined = df_combined_temp['title'] +" "+ df_combined_temp['content']
df_combined = df_combined.str.strip()
df_combined.replace('\s+', ' ', regex=True, inplace=True)
df_combined

0        aernout mik communitas aernout mik zaehlt zu d...
1        anton henning dt engl die vorliegende publikat...
2        die feuchtwangers familie tradition und jüdisc...
3        grüne philosophie ein konservativer denkansatz...
4        technische dokumentation im maschinen und anla...
                               ...                        
18138    analysis für informatiker grundlagen methoden ...
18139    der wasseratlas ein weltatlas zur wichtigsten ...
18140    grundkurs programmieren in java programmieren ...
18141    spiel mit der wirklichkeit zur entwicklung dok...
18142    dokument fälschung wirklichkeit materialband z...
Length: 18143, dtype: object

<h1>Remove Stop Words</h1>

In [25]:
from nltk.corpus import stopwords
stop = stopwords.words('german')
df['content_nltk'] = df_combined.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df['content_nltk'].iloc[3]

'grüne philosophie konservativer denkansatz umweltschutz liegt herkoemmlich hoheitsgebiet politischen linken bedrohung natur zufolge internationalen kapitalismus konsumverhalten unbegrenzten ausbeutung natuerlicher ressourcen zuzuschreiben roger scruton entbloesst grundverstaendnis gefaehrlichen trugschluss zerstoererisch kosysteme wirkt denen zukunft abhaengt konservative denker wuerdigt traditionelle prinzipien offensichtlichste wirksamste mittel erhalt planeten sichern lokalismus buergerverantwortung schlagen dabei bemuehungen schwerfaelliger nichtregierungsorganisationen internationaler komitees zukunft mitnichten gesichert roger scruton beweist gedankenreichen schrift fortbestand kosystems erde moeglich'

<h1>Tokenizing</h1>

In [26]:
# df_tok = df['content_nltk']
# df_tok

In [27]:
# tokenized = [ nltk.word_tokenize( str(sentence), language='german' ) for sentence in df_tok ]
# print(tokenized)

In [28]:
# df_tok = df.apply(lambda row: nltk.word_tokenize(row['content_nltk'], language='german'), axis=1)

In [29]:
#df_tok

In [30]:
# df_tok_test = df_tok[:3]
# df_tok_test

<h1>Lemmetization</h1>

In [31]:
import spacy
nlp = spacy.load("de_core_news_sm")

df["content_nltk"] = df["content_nltk"].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))

In [32]:
df["content_nltk"]

0        aernout mik communitas aernout mik zaehlt wich...
1        anton henning dt engl vorliegend publikation w...
2        feuchtwangers familie tradition jüdisch selbst...
3        grüne philosophie konservativ denkansatz umwel...
4        technische dokumentation maschinen anlagenbau ...
                               ...                        
18138    analysis informatiker grundlagen methoden algo...
18139    wasseratlas weltatlas wichtig ressource lebens...
18140    grundkurs programmieren java programmieren ler...
18141    spiel wirklichkeit entwicklung dokufiktionaler...
18142    dokument fälschung wirklichkeit materialband z...
Name: content_nltk, Length: 18143, dtype: object

<h1>Export</h1>

In [33]:
df.to_pickle('../data/after_nltk/data_after_nltk.pkl')
# df_content.to_csv('../data/after_nltk/df_content.csv', index=True)
#df["content_nltk"].to_csv('../data/after_nltk/df_content_after_nltk+titles.csv', index=True)
df["content_nltk"].to_csv('../data/after_nltk/df_content_after_nltk.csv', index=True)
# df.to_excel('../data/after_nltk/data_after_nltk.xlsx', index = False)

In [34]:
# #counts number of lines
# fname = "df_content.txt"
# count = 0
# with open(fname, 'r') as f:
#     for line in f:
#         count += 1
# print("Total number of lines is:", count)